$$
\newcommand{\d}[2]{d_{#1}^{#2}}
\newcommand{\p}[2]{p_{#1}^{#2}}
\newcommand{\ceil}[1]{\left \lceil #1 \right \rceil }
\newcommand{\floor}[1]{\left \lfloor #1 \right \rfloor }
\newcommand{\dl}[2]{d_{#1}^{#2'}}
\begin{alignat}{3}
&\text{minimizar }&f(\d1-,\dl2-,\dl2+,\d3+; \p1+=5, \p2-=4, \p2+=10, \p3-=5, t_1 = 125, t2 = 12, t_3 = 55)\\
&\text{sujeito a }\\
&&D_1 \leq 6 \\
&&D_2 \geq 2 \\
&&D_3 \geq 1 \\
&&12D_1+9D_2+5D_3+\d1-\geq125\\
&&D_1+\frac{2}{5}D_2+\frac{4}{5}D_3+\dl2--\dl2+=12\\
&&5D_1+5D_2+8D_3-\d3+ \leq 55\\
&&\d{i}{\pm} \geq 0\\
&&\dl2\pm \in \mathbb{N_0}
\end{alignat}
$$

In [ ]:
from pulp import *
from math import *

$$
\begin{align}
f(\mathbf{d^{\pm}};\mathbf{p^{\pm}}, \mathbf{t}) &= \sum \frac{p_i^+ d_i^+ + p_i^- d_i^-}{t_i} \\
f(\d1-,\dl2-,\dl2+,\d3+; \p1+=5, \p2-=4, \p2+=10, \p3-=5)
&= \frac{5\d1-}{125} + \frac{4\dl2- + 10\dl2+}{12} + \frac{5\d3+}{55}
\end{align}
$$

In [ ]:
d = {
    (i,pm):LpVariable(f"d{i}{'mais' if pm == '+' else 'menos'}", lowBound=0, cat = 'Integer' if i == 2 else 'Continuous') 
        for i in range(1,4) for pm in ["+", "-"]
}
D = {i: LpVariable(f"D{i}", lowBound=0) for i in range(1, 4)}

In [ ]:
modelo = LpProblem("modelo1", LpMinimize)
modelo += (D[1] <= 6, "maximo de D1")
modelo += (D[2] >= 2, "minimo de D2")
modelo += (D[3] >= 1, "minimo de D3")
modelo += (12*D[1] + 9*D[2] + 5*D[3]+d[1, "-"] >= 125, "meta de alcance de lucro")
modelo += (D[1] + (2/5) * D[2] + (4/5) * D[3] + d[2,"-"] - d[2,"+"] == (60/5), "meta de manter o nivel de mao de obra")
modelo += (5*D[1] + 5*D[2] + 8*D[3] - d[3, "+"] <= 55, "meta de travar o investimento")
modelo += (5*d[1, "-"])/125 + (4*d[2, "-"] + 10*d[2, "+"])/12 + (5*d[3, "+"])/55 #objectivo
modelo

modelo1:
MINIMIZE
0.04*d1menos + 0.8333333333333334*d2mais + 0.3333333333333333*d2menos + 0.09090909090909091*d3mais + 0.0
SUBJECT TO
maximo_de_D1: D1 <= 6

minimo_de_D2: D2 >= 2

minimo_de_D3: D3 >= 1

meta_de_alcance_de_lucro: 12 D1 + 9 D2 + 5 D3 + d1menos >= 125

meta_de_manter_o_nivel_de_mao_de_obra: D1 + 0.4 D2 + 0.8 D3 - d2mais + d2menos
 = 12

meta_de_travar_o_investimento: 5 D1 + 5 D2 + 8 D3 - d3mais <= 55

VARIABLES
D1 Continuous
D2 Continuous
D3 Continuous
d1menos Continuous
0 <= d2mais Integer
0 <= d2menos Integer
d3mais Continuous

In [ ]:
print("status: ",modelo.solve())
print("valor do objetivo:",value(modelo.objective))
print("variáveis:")
for i in modelo.variables():
    print("   ", i.name, "=", i.varValue)

status:  1
valor do objetivo: 1.9335664545454545
variáveis:
    D1 = 6.0
    D2 = 5.2692308
    D3 = 1.1153846
    d1menos = 0.0
    d2mais = 0.0
    d2menos = 3.0
    d3mais = 10.269231
